In [26]:
import numpy as np
import pandas as pd
import os
import pdb
from random import random
import math
import matplotlib.pyplot as plta
import pylab
import matplotlib.animation as animation
from scipy.spatial import distance
from scipy.spatial.distance import euclidean
from mpl_toolkits.mplot3d import Axes3D
def distance(p0, p1):
    return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2 +(p0[2] - p1[2])**2  )
def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy
def count_xtal_waters(ROI, df, radius):
    ref_waters_in_OEC = []
    OEC_ROI = radius
    for n,rew in df.iterrows():
        point1= [float(df.loc[n,'x']),float(df.loc[n,'y']),float(df.loc[n,'z'])]
        point2= [float(ROI['x'].values[0]),float(ROI['y'].values[0]),float(ROI['z'].values[0])]
        if ((distance(point1,point2) < OEC_ROI )):
            ref_waters_in_OEC.append(df.loc[n])
    return len(ref_waters_in_OEC)

In [27]:
colspecs_pdb = [(0, 6), (6, 11), (12, 16), (16, 17), (17, 20), (21, 22), (22, 26),
            (26, 27), (30, 38), (38, 46), (46, 54), (54, 60), (60, 66), (76, 78),
            (78, 80)]

names_pdb = ['ATOM', 'serial', 'name', 'altloc', 'resname', 'chainid', 'resseq',
         'icode', 'x', 'y', 'z', 'occupancy', 'tempfactor', 'element', 'charge']

pdb_path = 'files/peaks_SOL_rms2.69_7_13_22.pdb'
peaks = pd.read_fwf(pdb_path, names=names_pdb, colspecs=colspecs_pdb,skiprows=4)
peaks_in_roi = peaks.copy()
peaks

,ATOM,serial,name,altloc,resname,chainid,resseq,icode,x,y,z,occupancy,tempfactor,element,charge
0,ATOM,2327.0,O,NaN,HOH,X,1.0,NaN,81.653,108.127,20.046,10.81,10.81,O,NaN
1,ATOM,6062.0,O,NaN,HOH,X,2.0,NaN,84.839,43.055,55.808,10.62,10.62,O,NaN
2,ATOM,6398.0,O,NaN,HOH,X,3.0,NaN,93.976,20.540,58.748,10.54,10.54,O,NaN
3,ATOM,5787.0,O,NaN,HOH,X,4.0,NaN,80.996,33.221,53.062,10.50,10.50,O,NaN
4,ATOM,4618.0,O,NaN,HOH,X,5.0,NaN,28.257,175.076,39.768,10.49,10.49,O,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8282,ATOM,3153.0,O,NaN,HOH,X,8283.0,NaN,6.840,218.940,26.383,2.69,2.69,O,NaN
8283,ATOM,2979.0,O,NaN,HOH,X,8284.0,NaN,97.727,143.706,25.266,2.69,2.69,O,NaN
8284,ATOM,979.0,O,NaN,HOH,X,8285.0,NaN,67.252,163.799,7.752,2.69,2.69,O,NaN
8285,ATOM,2746.0,O,NaN,HOH,X,8286.0,NaN,62.696,159.396,23.082,2.69,2.69,O,NaN


In [28]:
gro_path = 'files/0F.pdb_fittedto55all.pdb' #Note that 0F structrue has extended chain G relative to LU34_0F
reference = pd.read_fwf(gro_path, names=names_pdb, colspecs=colspecs_pdb, skiprows=4)
ref2=reference[(reference['resname']=='HOH') | (reference['resname']=='OOO') | (reference['resname']=='HOH') | (reference['name']=='W1')|(reference['name']=='W2')|(reference['name']=='W3')|(reference['name']=='W4')]
df_water =ref2
df_water.head()

,ATOM,serial,name,altloc,resname,chainid,resseq,icode,x,y,z,occupancy,tempfactor,element,charge
2627,HETATM,2628.0,W1,NaN,OEC,A,601.0,NaN,83.512,35.203,52.954,1.0,27.44,O,NaN
2628,HETATM,2629.0,W2,NaN,OEC,A,601.0,NaN,81.644,37.556,53.164,1.0,29.43,O,NaN
2629,HETATM,2630.0,W3,NaN,OEC,A,601.0,NaN,83.046,40.226,53.709,1.0,28.51,O,NaN
2630,HETATM,2631.0,W4,NaN,OEC,A,601.0,NaN,85.200,42.041,55.104,1.0,31.98,O,NaN
16552,HETATM,16547.0,O,NaN,OOO,G,1.0,NaN,65.040,58.188,50.009,1.0,24.85,O,NaN


# ROI centered on OEC

In [29]:
radius=20
ROI_OEC1 = reference[(reference['resname'] =='OEC') & (reference['chainid'] =='A') ]
OEC=1
if OEC==1:
    ROI = ROI_OEC1
ROI

,ATOM,serial,name,altloc,resname,chainid,resseq,icode,x,y,z,occupancy,tempfactor,element,charge
2622,HETATM,2623.0,O1,NaN,OEC,A,601.0,NaN,84.177,39.939,58.171,1.0,24.17,O,NaN
2623,HETATM,2624.0,O2,NaN,OEC,A,601.0,NaN,85.184,38.154,57.178,1.0,25.85,O,NaN
2624,HETATM,2625.0,O3,NaN,OEC,A,601.0,NaN,83.321,37.504,58.528,1.0,26.77,O,NaN
2625,HETATM,2626.0,O4,NaN,OEC,A,601.0,NaN,84.595,36.107,55.291,1.0,28.57,O,NaN
2626,HETATM,2627.0,O5,NaN,OEC,A,601.0,NaN,82.937,38.071,55.842,1.0,25.86,O,NaN
2627,HETATM,2628.0,W1,NaN,OEC,A,601.0,NaN,83.512,35.203,52.954,1.0,27.44,O,NaN
2628,HETATM,2629.0,W2,NaN,OEC,A,601.0,NaN,81.644,37.556,53.164,1.0,29.43,O,NaN
2629,HETATM,2630.0,W3,NaN,OEC,A,601.0,NaN,83.046,40.226,53.709,1.0,28.51,O,NaN
2630,HETATM,2631.0,W4,NaN,OEC,A,601.0,NaN,85.200,42.041,55.104,1.0,31.98,O,NaN
2631,HETATM,2632.0,CA1,NaN,OEC,A,601.0,NaN,84.513,40.066,55.842,1.0,31.38,Ca,NaN


# List of all xtal waters in ROI

In [30]:
ref_waters_in_OEC = []
    
for n,rew in df_water.iterrows():
    point1= [float(df_water.loc[n,'x']),float(df_water.loc[n,'y']),float(df_water.loc[n,'z'])]
  
    for x in range(0,len(ROI)):
        point2= [float(ROI['x'].values[x]),float(ROI['y'].values[x]),float(ROI['z'].values[x])]
        if ((distance(point1,point2) < radius )):
            ref_waters_in_OEC.append(df_water.loc[n])
            break

ref_waters_in_OEC=pd.DataFrame(ref_waters_in_OEC)
print(len(ref_waters_in_OEC))


161


# Make list of ALL strong peaks in ROI

In [31]:
colspecs_pdb = [(0, 6), (6, 11), (12, 16), (16, 17), (17, 20), (21, 22), (22, 26),
            (26, 27), (30, 38), (38, 46), (46, 54), (54, 60), (60, 66), (76, 78),
            (78, 80)]

names_pdb = ['ATOM', 'serial', 'name', 'altloc', 'resname', 'chainid', 'resseq',
         'icode', 'x', 'y', 'z', 'occupancy', 'tempfactor', 'element', 'charge']



peaks_in_OEC=[]
for z,ruw in peaks_in_roi.iterrows():
    point1= [float(peaks_in_roi.loc[z,'x']),float(peaks_in_roi.loc[z,'y']),float(peaks_in_roi.loc[z,'z'])]
    
    for x in range(0,len(ROI)):
        point2= [float(ROI['x'].values[x]),float(ROI['y'].values[x]),float(ROI['z'].values[x])]

        if ((distance(point1,point2) < radius) ):
            peaks_in_OEC.append(peaks_in_roi.loc[z])
     
            break
peaks_in_OEC=pd.DataFrame(peaks_in_OEC)
print(len(peaks_in_OEC))

249


In [32]:
import Bio.PDB as bpdb
s = bpdb.PDBParser().get_structure('temp', 'files/peaks_SOL_rms2.69_7_13_22.pdb')

In [33]:
class ResSelect(bpdb.Select):
    def accept_residue(self, res):
        if res.id[1] in peaks_in_OEC['resseq'].values:
            return True
        else:
            return False

In [34]:
io = bpdb.PDBIO()
io.set_structure(s)
io.save('files/strong_peaks_in_20_angstrom_of_OEC_second.pdb', ResSelect())